## 初期設定

In [1]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
import src.config as cfg
import src.data_loader as dl
import src.cleaning_utils as cu
print("🔁 autoreload 有効化完了")
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict
import hashlib

🔁 autoreload 有効化完了


In [2]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

{'project_name': 'stock_screening',
 'data_path': {'raw': './data/raw',
  'interim': './data/interim',
  'processed': './data/processed'},
 'files': {'raw': ['fy-balance-sheet.csv',
   'fy-cash-flow-statement.csv',
   'fy-profit-and-loss.csv',
   'fy-stock-dividend.csv'],
  'reference': ['CodeData.csv'],
  'interim': None,
  'processed': None},
 'years': [2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021,
  2022,
  2023,
  2024,
  2025],
 'na_values': [''],
 'output': {'base_path': './output'}}

## 本ファイルの説明
データが最低限の品質基準を満たしているかを網羅的に確認し、ファイルロードからクリーニングまでのプロセスを決定します。

<details>
<summary><b>結果</b></summary>


</details>


### データのロードの確認
ファイル重複やファイル欠損、破壊がないかを確かめ、プロセスを確定します。
<details>
<summary><b>結果</b></summary>

ファイルの重複や欠損はなく、正しくロードされました。  
修正処理は不要です。
</details>

In [3]:
print("")
print("ファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"]
)
display(df_DATAs_BY_ALL_FILEs)

print("")
print("次に欠けてるファイルがないか確認します")
def chk_file_missing(df_DATAs_BY_ALL_FILEs):
    flg = True
    for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
        if df.empty:
            print("❌ ".filename,year,"データフレームが空です")
            flg = False
    if flg:
        print("✅ ファイル欠損はありません。")
chk_file_missing(df_DATAs_BY_ALL_FILEs)

print("")
print("重複ファイルがないか確認します")
def df_hash(df, ignore_order=False):
    data_only = df.copy().fillna("").astype(str)
    if ignore_order:
        # 行順も無視（全列ソートしてから比較）
        data_only = data_only.sort_values(by=data_only.columns.tolist()).reset_index(drop=True)
    # 列名を無視して内容だけでハッシュを生成
    h = hashlib.md5(pd.util.hash_pandas_object(data_only, index=True).values)
    return h.hexdigest()
hash_map = defaultdict(list)
for (fname, year), df in df_DATAs_BY_ALL_FILEs.items():
    h = df_hash(df, ignore_order=False)  # ←行順を無視したいなら True に
    hash_map[h].append((fname, year))
duplicates = {h: keys for h, keys in hash_map.items() if len(keys) > 1}
if duplicates:
    print("🔍 同じ内容のデータフレーム（ヘッダー無視）が見つかりました：\n")
    for h, key_group in duplicates.items():
        print(f"ハッシュ: {h}")
        for fname, year in key_group:
            print(f"  - {fname}（{year}）")
        print()
else:
    print("✅ 重複データフレームはありません。")

print("")
print("データロードの確認を終了します。")



ファイル毎にデータをロードします


{('fy-balance-sheet.csv',
  2010):        コード         年度           総資産          純資産         株主資本        利益剰余金  \
 0     1301 2010-03-01   64301000000  18538000000  18391000000  12589000000   
 1     1332 2010-03-01  383924000000  79567000000  69766000000  32530000000   
 2     1333 2010-03-01  113684000000  24435000000            -            -   
 3     1352 2010-03-01   11967000000   5439000000   5268000000   1561000000   
 4     1375 2010-03-01   32083000000   6318000000   6307000000   3056000000   
 ...    ...        ...           ...          ...          ...          ...   
 3401  9993 2010-03-01   39681154000  25865933000  25905452000  21401677000   
 3402  9994 2010-03-01   33044000000  15750000000  15730000000   6427000000   
 3403  9995 2010-03-01   36949822000  16216967000  16315762000   7883718000   
 3404  9996 2010-03-01   25279430000  17036692000  17124079000  14463977000   
 3405  9997 2010-03-01  119703000000  54217000000  55392000000  42578000000   
 
              短期


次に欠けてるファイルがないか確認します
✅ ファイル欠損はありません。

重複ファイルがないか確認します
✅ 重複データフレームはありません。

データロードの確認を終了します。


### 欠損値表現の確認と処理
欠損値の表現を確認し、修正プロセスを決定します。

<details>
<summary><b>結果</b></summary>

"-"が欠損値として使われています。  
これはsetting.yamlのnan_valuesに登録し欠損値として読み込みます。  
  
また、ゼロも使われています。  
ゼロは値として残し、必要に応じてfeatures_generatorで再計算し更新します。

以下がゼロが使われている項目です。  
{'fy-balance-sheet.csv': ['BPS', '短期借入金', '総資産', '自己資本比率', '長期借入金']  
'fy-cash-flow-statement.csv': ['営業CF', '営業CFマージン', '投資CF', '現金同等物', '設備投資', '財務CF']  
'fy-profit-and-loss.csv': ['EPS', 'ROA', 'ROE', '営業利益', '売上高', '純利益', '経常利益'],  
'fy-stock-dividend.csv': ['一株配当', '剰余金の配当', '純資産配当率', '総還元性向', '自社株買い', '配当性向']}
</details>

In [4]:
# 欠損値の表現確認
print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"]
)
#display(df_DATAs_BY_ALL_FILEs)
print("")
print("ファイルごとに各文字列の個数をカウントします")
COUNT_COLUMNS = ["col","empty", "space", "-", "―", "—", "--", "Na", "na", "N/A", "n/a", "None", "none", "NULL", "null", "0", "alphabet"]
FINAL_COLUMNS = COUNT_COLUMNS + ["ファイル名", "会計年"] # 追跡を明確にするため
df_placeholder_counts = pd.DataFrame(columns=FINAL_COLUMNS)
current_index = 0
for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
    for col in df:
        ser = df[col].fillna("")
        ser_str = ser.astype(str)
        #display(ser_str)
        row_data_tuple = (
            col,
            (ser_str == '').sum(), (ser_str == ' ').sum(),
            (ser_str == '-').sum(), (ser_str == '―').sum(), (ser_str == '—').sum(), (ser_str == '--').sum(),
            (ser_str == 'Na').sum(), (ser_str == 'na').sum(), (ser_str == 'N/A').sum(), (ser_str == 'n/a').sum(),
            (ser_str == 'None').sum(), (ser_str == 'none').sum(), (ser_str == 'NULL').sum(), (ser_str == 'null').sum(),
            (ser_str == '0').sum(), 
            ser_str.str.contains('[A-Za-z]', na=False).sum()
        )
        df_placeholder_counts.loc[current_index, COUNT_COLUMNS] = row_data_tuple
        df_placeholder_counts.loc[current_index, "ファイル名"] = filename
        df_placeholder_counts.loc[current_index, "会計年"] = year

        current_index += 1
display(df_placeholder_counts)



ファイル毎にデータを取得します

ファイルごとに各文字列の個数をカウントします


,col,empty,space,-,―,—,--,Na,na,N/A,n/a,None,none,NULL,null,0,alphabet,ファイル名,会計年
0,コード,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,fy-balance-sheet.csv,2010
1,年度,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,fy-balance-sheet.csv,2010
2,総資産,0,0,31,0,0,0,0,0,0,0,0,0,0,0,0,0,fy-balance-sheet.csv,2010
3,純資産,0,0,31,0,0,0,0,0,0,0,0,0,0,0,0,0,fy-balance-sheet.csv,2010
4,株主資本,0,0,159,0,0,0,0,0,0,0,0,0,0,0,0,0,fy-balance-sheet.csv,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,剰余金の配当,0,0,465,0,0,0,0,0,0,0,0,0,0,0,1,0,fy-stock-dividend.csv,2025
556,自社株買い,0,0,1385,0,0,0,0,0,0,0,0,0,0,0,8,0,fy-stock-dividend.csv,2025
557,配当性向,0,0,721,0,0,0,0,0,0,0,0,0,0,0,16,0,fy-stock-dividend.csv,2025
558,総還元性向,0,0,763,0,0,0,0,0,0,0,0,0,0,0,135,0,fy-stock-dividend.csv,2025


In [5]:
print("")
print("欠損値と疑われるコードを出力します。")
df = df_placeholder_counts.drop(["ファイル名","会計年"],axis=1)
df = df.groupby("col").sum().sum()
#display(df)

print("")
print("疑われるコードが含まれている列を出力します")
df = df_placeholder_counts.drop(["ファイル名","会計年"],axis=1)
df = df.groupby("col").sum().sum()
pl_list = df[df >0].index.tolist()
df = df_placeholder_counts.drop(["ファイル名","会計年"],axis=1)
for value in pl_list:
    list = df[df[value]>0]["col"].unique()
    #print("--------------",value,"のチェック----------------------------")
    #display(list)

print("")
print("アルファベットが含まれているコードを出力します。")
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    df_tmp = df[df["コード"].str.contains("[A-Za-z]", na=False)]
    code_list = df_tmp["コード"].to_list()
    #print(code_list)

print("")
print("ゼロが登録されているファイルと列を出力します。")
df = df_placeholder_counts
df_0 = df.groupby(["ファイル名","col"])["0"].sum()
df_0 = df_0[df_0>0].reset_index()
print(df_0)
print(df_0['col'].dtype)

# 'col' 列のすべての要素のデータ型を取得し、その出現回数をカウント
#type_counts = df_0['col'].apply(type).value_counts()
#print("--- 'col' 列の要素のデータ型カウント ---")
#print(type_counts)

print("")
print("4つのファイルごとに該当列のリストを準備します")
#df_0['col'] = df_0['col'].astype(str) 
#df_0['col'] = df_0['col'].apply(lambda x: str(x) if x is not None else x)
df_temp = df_0[['ファイル名', 'col']]
grouped = df_temp.groupby('ファイル名')
col_lists = {
    filename: group['col'].tolist()  # 各グループから 'col' 列を抽出し、tolist()でPythonリストに変換
    for filename, group in grouped
}
print(col_lists)

print("")
print("ゼロの含まれるコードのデータを確認します。")
print("まずは年でわかれた各ファイルをつなぎ合わせます")
df_merge = {}
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    if filename not in df_merge:
        df_merge[filename] = []
    df_merge[filename].append(df)
df_final = {}
for filename in df_merge:
    df_final[filename]  = pd.concat(df_merge[filename])
display(df_final["fy-balance-sheet.csv"])

print("")
print("ファイルごとにゼロをもつコードの値をチェックします。")
filename = "fy-profit-and-loss.csv"
print("ファイル名:",filename)
df_all = df_final[filename]
for col_value in col_lists[filename]:
    #print(filename, col)
    df = df_all[df_all[col_value] == "0"]
    code_list = df["コード"].unique()
    print("-----------------------",col_value,"-----------------------")
    for code in code_list:
        display(df_all[df_all["コード"] == code])


欠損値と疑われるコードを出力します。

疑われるコードが含まれている列を出力します

アルファベットが含まれているコードを出力します。

ゼロが登録されているファイルと列を出力します。
                         ファイル名       col     0
0         fy-balance-sheet.csv       BPS     5
1         fy-balance-sheet.csv     短期借入金    12
2         fy-balance-sheet.csv       総資産     1
3         fy-balance-sheet.csv    自己資本比率     1
4         fy-balance-sheet.csv     長期借入金    17
5   fy-cash-flow-statement.csv      営業CF     2
6   fy-cash-flow-statement.csv  営業CFマージン    21
7   fy-cash-flow-statement.csv      投資CF    15
8   fy-cash-flow-statement.csv     現金同等物     2
9   fy-cash-flow-statement.csv      設備投資   234
10  fy-cash-flow-statement.csv      財務CF    11
11      fy-profit-and-loss.csv       EPS    12
12      fy-profit-and-loss.csv       ROA    13
13      fy-profit-and-loss.csv       ROE     6
14      fy-profit-and-loss.csv      営業利益     2
15      fy-profit-and-loss.csv       売上高    11
16      fy-profit-and-loss.csv       純利益     2
17      fy-profit-and-loss.csv      経常利益     4
18       fy-s

,コード,年度,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率
0,1301,2010-03-01,64301000000,18538000000,18391000000,12589000000,26724000000,2712000000,1651.87,28.1
1,1332,2010-03-01,383924000000,79567000000,69766000000,32530000000,114639000000,106993000000,231.06,16
2,1333,2010-03-01,113684000000,24435000000,-,-,-,-,-,21.5
3,1352,2010-03-01,11967000000,5439000000,5268000000,1561000000,-,497000000,628.95,44
4,1375,2010-03-01,32083000000,6318000000,6307000000,3056000000,-,7786000000,1.62,19
...,...,...,...,...,...,...,...,...,...,...
4139,9993,2025-02-01,55577000000,25969000000,25830000000,21459000000,8100000000,3620000000,2407.43,46.7
4140,9994,2025-03-01,66942000000,37330000000,34724000000,25670000000,3100000000,2968000000,3251.97,52.7
4141,9995,2024-03-01,49059000000,26593000000,24580000000,13707000000,11524000000,-,901.34,54.2
4142,9996,2025-03-01,35597241000,26268555000,26166862000,24318790000,683000000,-,3149.63,73.8



ファイルごとにゼロをもつコードの値をチェックします。
ファイル名: fy-profit-and-loss.csv
----------------------- EPS -----------------------


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
2510,7847,2010-02-01,2629009000,-14942000,-2665000,-266854000,0,-,-
2612,7847,2011-02-01,3542672000,508957000,510613000,263862000,0,22.05,7.7
2680,7847,2012-02-01,4055008000,843361000,846047000,724075000,0,37.67,18.03
2768,7847,2013-02-01,3462211000,529420000,613467000,1229771000,177.06,39.4,28.82
2864,7847,2014-02-01,4523838000,1249910000,1358987000,1161089000,167.17,27.79,21.02
2968,7847,2015-02-01,4024300000,934694000,1032517000,673754000,102.85,15.93,12.03
3057,7847,2016-02-01,2770981000,62146000,47683000,16131000,2.51,0.39,0.32
3152,7847,2017-02-01,2946308000,326286000,362631000,248072000,38.58,5.86,4.62
3235,7847,2018-02-01,2921617000,414215000,400213000,257889000,40.11,5.91,4.82
3300,7847,2019-02-01,2965753000,356995000,376597000,249625000,38.82,5.61,4.57


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
2592,7961,2010-03-01,9639000000,24000000,12000000,0,0,0,0
2694,7961,2011-03-01,10128000000,88000000,81000000,-272000000,-65.55,-,-
2763,7961,2012-03-01,10893000000,426000000,421000000,320000000,77.12,9.48,4.11
2850,7961,2013-03-01,10461000000,572000000,574000000,533000000,128.46,13.57,7.17
2947,7961,2014-03-01,12394000000,780000000,804000000,998000000,240.71,20.18,12.73
3051,7961,2015-03-01,12158000000,706000000,725000000,509000000,122.77,9.51,6.02
3140,7961,2016-03-01,11598000000,747000000,821000000,491000000,118.44,8.41,5.5
3235,7961,2017-03-01,12967000000,956000000,1034000000,669000000,161.38,10.27,6.75
3317,7961,2018-03-01,13151000000,989000000,1062000000,764000000,184.31,10.88,7.3
3382,7961,2019-03-01,12961000000,845000000,935000000,620000000,149.58,8.31,5.7


...

----------------------- 経常利益 -----------------------


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
1934,6492,2010-11-01,12649948000,1579759000,1729536000,1018176000,567.86,12.05,8.4
2018,6492,2011-11-01,10195847000,803624000,939475000,384559000,214.48,4.44,3.45
2063,6492,2012-11-01,8347631000,338183000,458348000,197148000,109.95,2.27,1.85
2116,6492,2013-11-01,7404326000,134070000,304043000,187500000,106.12,2.13,1.68
2170,6492,2014-11-01,7437199000,159981000,351872000,197189000,112.1,2.22,1.83
2230,6492,2015-11-01,7624708000,373307000,615591000,342001000,196.97,3.76,3.07
2293,6492,2016-11-01,7501002000,113623000,419623000,226304000,130.36,2.46,2.02
2362,6492,2017-11-01,8304000000,234430000,373000000,193000000,111.19,2.05,1.37
2437,6492,2018-11-01,7577000000,-67930000,0,165000000,95.07,1.73,1.19
2497,6492,2019-11-01,6663000000,-872933000,-792000000,-983000000,-567.26,-,-


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
2950,8836,2010-03-01,2057000000,23000000,49000000,55000000,0.82,1.45,1.09
3054,8836,2011-03-01,564000000,-98000000,-102000000,-341000000,-5.08,-,-
3123,8836,2012-03-01,380000000,-89000000,-45000000,-40000000,-0.6,-,-
3210,8836,2013-03-01,560000000,51000000,92000000,94000000,1.41,2.76,2.28
3307,8836,2014-03-01,592000000,58000000,73000000,74000000,1.57,2.25,1.89
3411,8836,2015-03-01,424000000,14000000,26000000,29000000,0.62,0.88,0.75
3500,8836,2016-03-01,447000000,123000000,115000000,71000000,1.39,2.1,1.79
3595,8836,2017-03-01,246000000,-162000000,-161000000,-162000000,-1.69,-,-
3677,8836,2018-03-01,515000000,-138000000,-143000000,179000000,1.86,8.61,7.12
3739,8836,2019-03-01,141000000,-103000000,-107000000,-109000000,-1.14,-,-


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
1656,6029,2010-12-01,603562000,-,-85121000,-98762000,-17.33,-,-
1721,6029,2011-12-01,710741000,-,-5655000,-21465000,-3.77,-,-
1761,6029,2012-12-01,772046000,-24896000,-41981000,-44059000,-7.73,-,-
1812,6029,2013-12-01,1320152000,144316000,144549000,145542000,25.53,343.79,14.76
1862,6029,2014-12-01,1490027000,78002000,68673000,27604000,4.76,5.05,1.7
1919,6029,2015-12-01,2564070000,387415000,453313000,268734000,34.45,32.97,11.21
1981,6029,2016-12-01,3251564000,293905000,301387000,148703000,18.41,10.06,5.32
2048,6029,2017-12-01,3759158000,435573000,455475000,301430000,34.59,17.19,8.51
2120,6029,2018-12-01,3479200000,164280000,163697000,269930000,30.88,13.5,6.01
2180,6029,2019-12-01,2833781000,25196000,31411000,-146664000,-16.71,-,-


,コード,年度,売上高,営業利益,経常利益,純利益,EPS,ROE,ROA
1109,4314,2010-12-01,104331000000,-1800000000,-24596000000,-2738000000,-1748.21,-,-
1169,4314,2011-12-01,33757000000,1470000000,-8769000000,6900000000,4405.64,-95.86,1.5
1206,4314,2012-12-01,14561000000,3541000000,-3558000000,1990000000,1270.61,-38.62,0.47
1229,4314,2013-12-01,202912000000,-31301000000,-44218000000,1701000000,1086.09,-50.53,1.21
1247,4314,2014-12-01,112735000000,1042000000,-48000000,6151000000,3927.41,216.28,63.03
1281,4314,2015-12-01,1475000000,121000000,-54000000,-2000000,-1.28,-,-
1318,4314,2016-12-01,7700000000,2141000000,1811000000,1792000000,1144.19,40.81,21.03
1340,4314,2017-12-01,1724000000,-224000000,-398000000,-257000000,-164.09,-,-
1364,4314,2018-12-01,1430000000,94000000,-300000000,-410000000,-261.78,-,-
1406,4314,2019-12-01,1049000000,-284000000,-621000000,-946000000,-604.02,-,-


### データ型の整合性確認

<details>
<summary><b>結果</b></summary>

</details>

### 初期品質の確認

データのインデックスキーの役割をもつ以下の値を確認し、必要な処理のプロセスを決定します。


<summary><b>結果</b></summary>

</details>


In [ ]:
# 財務データ 結合前
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 60)

def get_file_info(df_datas_by_all_files):
    df_file_info = pd.DataFrame(columns=["file","year","code_counts","year_counts","column_counts"])
    for file, year in df_datas_by_all_files.keys():
        df = df_datas_by_all_files[(file,year)]
        # len(df_file_info)を次の新しい行のインデックスとして明示的に指定します。
        df_file_info.loc[len(df_file_info)] = [
            file, year, df["コード"].nunique(), df["年度"].nunique(),df.columns.nunique()
        ]
    return df_file_info

print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"],["-","","0"]
)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("企業コード数、エンドの数、列数をファイルごとにまとめます。")
df_file_info = get_file_info(df_DATAs_BY_ALL_FILEs)
#display(df_file_info)

print("")
print("各ファイルの年推移を可視化します。")
"""fig = px.line(df_file_info,x="year",y="code_counts",color="file")
fig.show()
fig = px.line(df_file_info,x="year",y="year_counts",color="file")
fig.show()
fig = px.line(df_file_info,x="year",y="column_counts",color="file")
fig.show()"""

print("")
print("列数はOKです")
print("最新年のファイルに登録されている年度を表示します。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df["年度"].unique())

print("")
print("最新年のファイルに過去のデータがあります。")
print("最新年に登録されている過去の年度と過去のデータが同じか確認します")
print("まずは、年度を限定して、コードの一覧を見ます")
#display(df[df["年度"]=="2024/12"])

print("")
print("コードを限定し同じかどうか調べます。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("違っています。これは更新データと思われます。")
print("最新年度にある過去のデータを取得し、過去のデータを更新します")
df_DATAs_BY_ALL_FILEs = dl.update_duplicated(df_DATAs_BY_ALL_FILEs, 2025)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("正しく処理が行われ、最新年に登録されている過去の年度と過去のデータが同じか確認します。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("最後に最新年度にある過去のデータを消去します。")
cutoff_date = pd.to_datetime("2025-01-01")
for file, year in df_DATAs_BY_ALL_FILEs.keys():
    if year == 2025:
        df = df_DATAs_BY_ALL_FILEs[(file,year)]
        df = df[df["年度"] >= cutoff_date]
        df_DATAs_BY_ALL_FILEs[(file,year)] = df
print("")
print("各ファイルの年推移をもう一度可視化し、改善していることを確かめます。") 
df_file_info_after = get_file_info(df_DATAs_BY_ALL_FILEs)
df_file_info_after["区分"] = "処理後"
df_file_info_after = df_file_info_after.set_index(["file", "year"]).sort_index()
df_file_info["区分"] = "処理前"
df_file_info = df_file_info.set_index(["file", "year"]).sort_index()
df_compare = pd.concat([df_file_info,df_file_info_after]).sort_index().reset_index()
#display(df_compare)
fig = px.line(df_compare,x="year",y="code_counts",color="区分")
fig.show()


In [ ]:
# 財務データ ファイルごとの結合
df_DATAs_BY_FILEs = dl.load_yearly_data(settings["data_path"], settings["years"], settings["files"],)
target_file = settings["files"][0]
display(df_DATAs_BY_FILEs[target_file][df_DATAs_BY_FILEs[target_file]["コード"]=="130A"])

In [ ]:
# 財務データ　全結合
df_ALL_DATAs = dl.merge_all_data(df_DATAs_BY_FILEs)
print(df_ALL_DATAs.shape)
#display(df_ALL_DATAs)
display(df_ALL_DATAs["コード"].unique())
display(df_ALL_DATAs["年度"].nunique())

In [ ]:
# 上場企業の情報
df = dl.load_code_list_info(settings["data_path"], settings["files_reference"][0])
print(settings["files_reference"][0], df.shape)
display(df)